In [ ]:
import pandas as pd
from nltk.corpus import stopwords
import re
from tqdm import tqdm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn import utils
from stop_words import get_stop_words

In [ ]:
data_df = pd.read_json('data/hackathon_dataset.json', lines=True)
# data_df.shape
# data_df.head(10)

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@;&=-]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z .,]')
STOPWORDS = set(get_stop_words('dutch'))

def clean_text(text, url=False):
    try:
        text = text.lower() # lowercase text
        text = text.replace('\n', ' ') 

        if url:
            text = text.split('?', 1)[0]

        text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
        text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are not in BAD_SYMBOLS_RE from text

        text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
        text = ' '.join(word.rstrip(",.") for word in text.split())
        text = ' '.join(word for word in text.split() if 1 < len(word) )
        text = re.sub(r'\b[a-zA-Z]\b', '', text) # remove one words 
        text = re.sub(r'http\S+', '', text) # remove links
        text = re.sub(' +', ' ', text) # remove exess spaces
        text = text.strip()
        if url:
            text = text.rsplit(' ', 1)[0]
            spl = text.split(' ', 1)
            if len(spl) > 1:
                text = spl[1]
            else:
                text = ''
        text = re.sub(r'\d{5,}', '', text) # digits len > 5 filterd out
        text = text.strip()
    except:
        print(text)
    return text

In [ ]:
def createDF(df):
    df_ = df[['DALTIX_ID','DESCRIPTION','DISPLAY_URL','NAME','SPECIFICATIONS']].astype(str)
    df_['DESCRIPTION'] = df_['DESCRIPTION'].apply(clean_text)
    df_['DISPLAY_URL'] = df_['DISPLAY_URL'].apply(clean_text, url=True)
    df_['NAME'] = df_['NAME'].apply(clean_text)
    df_['SPECIFICATIONS'] = df_['SPECIFICATIONS'].apply(clean_text)
    return df_

In [ ]:
df = createDF(data_df)
df['SPECIFICATIONS'].replace('none', '', inplace=True)
df['DESCRIPTION'].replace('none', '', inplace=True)

In [ ]:
df["text"] = df['DESCRIPTION'].map(str) + ' ' + df['DISPLAY_URL'].map(str)+ ' ' + df['NAME'].map(str)+ ' ' + df['SPECIFICATIONS'].map(str)

In [ ]:
df[["DALTIX_ID","text"]].to_csv("dataset_cleaned.csv", index=False)